In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import  LabelEncoder,OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
df=pd.read_csv('creditScoreClassification.csv')

C:\Users\ujjwa_n18433z\AppData\Local\Temp\ipykernel_18360\1528044309.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('creditScoreClassification.csv')


In [3]:
# 4
df['Age'] = df['Age'].replace({'invalid': None})
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df.loc[(df['Age'] < 0) | (df['Age'] > 120), 'Age'] = None
median_age = df['Age'].median()
df['Age'] = df['Age'].fillna(median_age)

In [4]:
df['Occupation'] = df['Occupation'].replace('_', np.nan)
df['Occupation'] = df.groupby('Name')['Occupation'].ffill()
df['Occupation'] = df['Occupation'].fillna(method='bfill')
df = df.reset_index(drop=True)

C:\Users\ujjwa_n18433z\AppData\Local\Temp\ipykernel_18360\300112647.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Occupation'] = df['Occupation'].fillna(method='bfill')


In [5]:
df['Annual_Income'] = df['Annual_Income'].astype(str).str.replace('_', '', regex=True)
df['Annual_Income'] = pd.to_numeric(df['Annual_Income'], errors='coerce')
df['Annual_Income'] = df['Annual_Income'].apply(lambda x: np.nan if x <= 0 else x)
df = df.reset_index(drop=True)
df['Annual_Income'].value_counts()

Annual_Income
17816.75       16
22434.16       16
40341.16       16
17273.83       16
109945.32      16
               ..
17079092.00     1
1910572.00      1
20179076.00     1
7980216.00      1
8299495.00      1
Name: count, Length: 13487, dtype: int64

In [6]:
df['Monthly_Inhand_Salary'] = df['Monthly_Inhand_Salary'].astype(str).str.replace('_', '', regex=True)
df['Monthly_Inhand_Salary'] = pd.to_numeric(df['Monthly_Inhand_Salary'], errors='coerce')
df['Monthly_Inhand_Salary'] = df['Monthly_Inhand_Salary'].apply(lambda x: np.nan if x <= 0 else x)
df['Monthly_Inhand_Salary'] = df.groupby('Name')['Monthly_Inhand_Salary'].transform(lambda x: x.fillna(x.mean()))
df['Monthly_Inhand_Salary'] = df['Monthly_Inhand_Salary'].fillna(df['Monthly_Inhand_Salary'].mean())

In [7]:
df['Outstanding_Debt'] = df['Outstanding_Debt'].replace({'nan': None, '!': None, '_': None})
df['Outstanding_Debt'] = pd.to_numeric(df['Outstanding_Debt'], errors='coerce')
median_value = df['Outstanding_Debt'].median()
df['Outstanding_Debt'] = df['Outstanding_Debt'].fillna(median_value)

In [8]:
df['Num_of_Loan'] = df['Num_of_Loan'].replace({'nan': None, '!': None, '_': None})
df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce')
median_value = df['Num_of_Loan'].median()
df['Num_of_Loan'] = df['Num_of_Loan'].fillna(median_value)

In [9]:
df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].replace({'nan': None, '!': None, '_': None})
df['Changed_Credit_Limit'] = pd.to_numeric(df['Changed_Credit_Limit'], errors='coerce')
median_value = df['Changed_Credit_Limit'].median()
df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].fillna(median_value)

In [10]:
df['Total_EMI_per_month']=df['Total_EMI_per_month'].astype('float32')

In [11]:
def extract_float(value):
    match = re.match(r'[-+]?\d*\.\d+|[-+]?\d+', str(value)) 
    return float(match.group()) if match else None

def extract_monthly_balance(df, column_name='Monthly_Balance'):
    extracted_balance = df[column_name].apply(extract_float).astype('float32')
    return extracted_balance
df['Monthly_Balance']=extract_monthly_balance(df)

In [12]:
df['Amount_invested_monthly']=extract_monthly_balance(df)
df['Payment_of_Min_Amount']=extract_monthly_balance(df)

In [13]:
def extract_years_months(input_string):
    if pd.isna(input_string):
        return 0, 0  
    input_string = str(input_string) 
    year_match = re.search(r'(\d+)\s*Years?', input_string)
    month_match = re.search(r'(\d+)\s*Months?', input_string)

    years = int(year_match.group(1)) if year_match else 0
    months = int(month_match.group(1)) if month_match else 0
    
    return years, months
def extract_years_months_from_column(df, column_name):



    extracted = df[column_name].apply(extract_years_months)
    
    years_series = extracted.apply(lambda x: x[0])
    months_series = extracted.apply(lambda x: x[1])
    
    return years_series, months_series
years, months = extract_years_months_from_column(df, 'Credit_History_Age')

In [14]:
df['Credit_History_Age_Year']=years
df['Credit_History_Age_month']=months

In [15]:
df=df.drop(['Credit_History_Age','SSN','ID', 'Customer_ID','Name','Type_of_Loan'],axis=1)

In [16]:
def convert_object_columns_to_float(df):
    object_cols = [ 'Age',  'Annual_Income', 'Monthly_Inhand_Salary',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
        'Delay_from_due_date', 'Num_of_Delayed_Payment',
       'Changed_Credit_Limit', 'Num_Credit_Inquiries', 
       'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Payment_of_Min_Amount',
       'Total_EMI_per_month', 'Amount_invested_monthly',
       'Monthly_Balance',  'Credit_History_Age_Year',
       'Credit_History_Age_month']
    
    for col in object_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        median_value = df[col].median()
        df[col] = df[col].fillna(median_value)
    return df
df=convert_object_columns_to_float(df)

In [17]:
df.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,...,Outstanding_Debt,Credit_Utilization_Ratio,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Credit_History_Age_Year,Credit_History_Age_month
0,January,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,3,...,809.98,26.822620,312.494080,49.574947,312.494080,High_spent_Small_value_payments,312.494080,Good,22,1
1,February,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,-1,...,809.98,31.944960,284.629150,49.574947,284.629150,Low_spent_Large_value_payments,284.629150,Good,0,0
2,March,33.0,Scientist,19114.12,1824.843333,3,4,3,4.0,3,...,809.98,28.609352,331.209869,49.574947,331.209869,Low_spent_Medium_value_payments,331.209869,Good,22,3
3,April,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,5,...,809.98,31.377862,223.451309,49.574947,223.451309,Low_spent_Small_value_payments,223.451309,Good,22,4
4,May,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,6,...,809.98,24.797347,341.489227,49.574947,341.489227,High_spent_Medium_value_payments,341.489227,Good,22,5


In [18]:
columns_to_encode = ['Payment_Behaviour', 'Occupation', 'Month','Credit_Mix']
 
label_encoder = LabelEncoder()

for column in columns_to_encode:
    df[column] = label_encoder.fit_transform(df[column])

print(df)

       Month   Age  Occupation  Annual_Income  Monthly_Inhand_Salary  \
0          3  23.0          12       19114.12            1824.843333   
1          2  23.0          12       19114.12            1824.843333   
2          6  33.0          12       19114.12            1824.843333   
3          0  23.0          12       19114.12            1824.843333   
4          7  23.0          12       19114.12            1824.843333   
...      ...   ...         ...            ...                    ...   
99995      0  25.0           9       39628.99            3359.415833   
99996      7  25.0           9       39628.99            3359.415833   
99997      5  25.0           9       39628.99            3359.415833   
99998      4  25.0           9       39628.99            3359.415833   
99999      1  25.0           9       39628.99            3359.415833   

       Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  Num_of_Loan  \
0                      3                4              3      

In [19]:
df.columns

Index(['Month', 'Age', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Payment_of_Min_Amount',
       'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour',
       'Monthly_Balance', 'Credit_Score', 'Credit_History_Age_Year',
       'Credit_History_Age_month'],
      dtype='object')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  int32  
 1   Age                       100000 non-null  float64
 2   Occupation                100000 non-null  int32  
 3   Annual_Income             100000 non-null  float64
 4   Monthly_Inhand_Salary     100000 non-null  float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  float64
 9   Delay_from_due_date       100000 non-null  int64  
 10  Num_of_Delayed_Payment    100000 non-null  float64
 11  Changed_Credit_Limit      100000 non-null  float64
 12  Num_Credit_Inquiries      100000 non-null  float64
 13  Credit_Mix                100000 non-null  in

In [21]:
X = df.drop('Credit_Score', axis=1)  # Replace 'target_column' with the actual 
y = df['Credit_Score']

In [22]:
# Label Encoding The target column 
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
param_grid_rf = {
    'n_estimators': [200,300],
    'max_depth': [ 5,30],
    'min_samples_split': [ 10,20]
}

In [25]:
grid_search = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, n_jobs=-1, verbose=2)

In [26]:
grid_search.fit(X_train, y_train)

# Best parameters found by Grid Search
print(f"Best Parameters: {grid_search.best_params_}")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters: {'max_depth': 30, 'min_samples_split': 10, 'n_estimators': 300}


In [27]:
y_pred = grid_search.predict(X_test)

# Evaluating the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")

Accuracy: 0.76935
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.70      0.70      3527
           1       0.77      0.76      0.76      5874
           2       0.79      0.80      0.80     10599

    accuracy                           0.77     20000
   macro avg       0.75      0.75      0.75     20000
weighted avg       0.77      0.77      0.77     20000

